In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
@import url('http://fonts.googleapis.com/css?family=Crimson+Text');
@import url('http://fonts.googleapis.com/css?family=Source+Code+Pro');

/* Change code font */
.CodeMirror pre {
    font-family: 'Input Mono Narrow', 'Source Code Pro', Consolas, monocco, monospace;
}

div.input_area {
    border-color: rgba(0,0,0,0.10);
}

div.text_cell {
    max-width: 105ex; /* instead of 100%, */
}

div.text_cell_render {
    font-family: "Crimson Text";
    font-size: 12pt;
    line-height: 145%; /* added for some line spacing of text. */
}

div.text_cell_render h1,
div.text_cell_render h2,
div.text_cell_render h3,
div.text_cell_render h4,
div.text_cell_render h5,
div.text_cell_render h6 {
    font-family: 'Crimson Text';
}

.rendered_html pre,
.rendered_html code {
    font-size: medium;
}

.rendered_html ol {
    list-style:decimal;
    margin: 1em 2em;
}

.prompt.input_prompt {
    color: rgba(0,0,0,0.5);
}

.cell.command_mode.selected {
    border-color: rgba(0,0,0,0.1);
}

.cell.edit_mode.selected {
    border-color: rgba(0,0,0,0.15);
    box-shadow: 0px 0px 5px #f0f0f0;
    -webkit-box-shadow: 0px 0px 5px #f0f0f0;
}

div.output_scroll {
    -webkit-box-shadow: inset 0 2px 8px rgba(0,0,0,0.1);
    box-shadow: inset 0 2px 8px rgba(0,0,0,0.1);
    border-radious: 2px;
}

#menubar .navbar-inner {
    -webkit-box-shadow: none;
    box-shadow: none;
    border-radius: 0;
    border: none;
    font-family: lato;
    font-weight: 400;
}

.navbar-fixed-top .navbar-inner,
.navbar-static-top .navbar-inner {
    box-shadow: none;
    -webkit-box-shadow: none;
    border: none;
}

div#notebook_panel {
    box-shadow: none;
    -webkit-box-shadow: none;
    border-top: none;
}

div#notebook {
    border-top: 1px solid rgba(0,0,0,0.15);
}

/* 
    This is a lazy fix, we *should* fix the 
    background for each Bootstrap button type
*/
#site * .btn {
    -webkit-box-shadow: none;
    box-shadow: none;
}


span.ansiblack {color: #073642;}
span.ansiblue {color: #2aa198;}
span.ansigray {color: #839496;}
span.ansigreen {color: #859900;}
span.ansipurple {color: #6c71c4;}
span.ansired {color: #dc322f;}
span.ansiyellow {color: #b58900;}

div.output_stderr {background-color: #dc322f;}
div.output_stderr pre {color: #eee8d5;}

.cm-s-ipython.CodeMirror {background: #fdf6e3; color: #073642;}
.cm-s-ipython div.CodeMirror-selected {background: #eee8d5 !important;}
.cm-s-ipython .CodeMirror-gutters {background: #fdf6e3; border-right: 0px;}
.cm-s-ipython .CodeMirror-linenumber {color: #839496;}
.cm-s-ipython .CodeMirror-cursor {border-left: 1px solid #657b83 !important;}

.cm-s-ipython span.cm-comment {color: #d33682;}
.cm-s-ipython span.cm-atom {color: #6c71c4;}
.cm-s-ipython span.cm-number {color: #6c71c4;}

.cm-s-ipython span.cm-property, .cm-s-ipython span.cm-attribute {color: #859900;}
.cm-s-ipython span.cm-keyword {color: #dc322f;}
.cm-s-ipython span.cm-string {color: #b58900;}
.cm-s-ipython span.cm-operator {color: #d33682;}
.cm-s-ipython span.cm-builtin {color: #6c71c4;}

.cm-s-ipython span.cm-variable {color: #859900;}
.cm-s-ipython span.cm-variable-2 {color: #268bd2;}
.cm-s-ipython span.cm-def {color: #cb4b16;}
.cm-s-ipython span.cm-error {background: #dc322f; color: #657b83;}
.cm-s-ipython span.cm-bracket {color: #586e75;}
.cm-s-ipython span.cm-tag {color: #dc322f;}
.cm-s-ipython span.cm-link {color: #6c71c4;}

.cm-s-ipython .CodeMirror-matchingbracket { text-decoration: underline; color: #073642 !important;}
th, td { 
      font-size: 18px;
      border-collapse: collapse;
      border-width:3px;
    }
    </style>

""")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from sklearn.preprocessing import RobustScaler, QuantileTransformer
import rasterio
from matplotlib import colors
from scipy import stats
from tqdm import *
from pyproj import Transformer
import ast
import glob
from io import StringIO

| KPI Name                  | KPI Interpretation                                                   | Public Data Source         |
|---------------------------|----------------------------------------------------------------------|----------------------------|
| flood_score               | Calculated score based on survey response                            | CDP Survey                 |
| insured_locations         | # of NFIP insured locations (should be indication of exposure)       | FEMA NFIP                  |
| fema_flood_events         | # of FEMA disaster declarations near city related to floods          | FEMA Disaster Declarations |
| fema_coastal_storms       | # of FEMA disaster declarations near city related to coastal storms  | FEMA Disaster Declarations |
| fema_landslides           | # of FEMA disaster declarations near city related to landslides      | FEMA Disaster Declarations |
| all_locs_building_grounds | Ratio of building grounds workers in city compared to rest of the US | OECS                       |
| all_locs_farming          | Ratio of farming/ag workers in city compared to rest of the US       | OECS                       |
| percent_built_before_1999 | Percentage of buildings built before 1999                            | US Census                  |
| coastal_flood_eals        | Coastal flood expected annual loss                                   | FEMA National Risk Index   |
| river_flood_eals          | River flood expected annual loss                                     | FEMA National Risk Index   |
| landslide_eals            | Landslide expected annual loss                                       | FEMA National Risk Index   |

In [ ]:
path = "../input/externald/Supplementary Data/" 
vulnerability = pd.read_csv(path + "cities_updated_geo_us_2020.csv")
acct_number = vulnerability["Account Number"]
counties = (vulnerability['counties'])
counties_weights = (vulnerability['counties_weights'])
states = (vulnerability['state'])
flag_multi_counties = vulnerability['flag_multiple_counties']
match_fips = vulnerability['FIPS']
MSA = (vulnerability['MSA'])
match_fips = [ast.literal_eval(match_fips[i]) if flag_multi_counties[i] else int(match_fips[i]) for i in range(len(match_fips))]
cities = vulnerability["city_bing"]
population = vulnerability["Population"]

In [ ]:
def standardize_rank(arr, direction=1):
    rank = (stats.mstats.rankdata(np.ma.masked_invalid(arr)))
    rank[rank == 0] = np.nan
    if(direction == 1):
        return rank/(np.nanmax(rank)) 
    else:
        return 1-(rank/(np.nanmax(rank)))


In [ ]:
cities_2020 = pd.read_csv("/kaggle/input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2020_Full_Cities_Dataset.csv")

cities_disc_2020 = pd.read_csv("/kaggle/input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/2020_Cities_Disclosing_to_CDP.csv")

cities_2020_merged = pd.merge(cities_2020, cities_disc_2020, on="Account Number", how="outer", copy=False)
unique_20 = np.unique(cities_disc_2020[cities_disc_2020["Country"] == "United States of America"]["Account Number"])
cities_2020_merged_sub = cities_2020_merged[np.in1d(cities_2020_merged["Account Number"], unique_20)]

def fetch_answer_individual(account, question_number, org_type="city", year=2018, corp_res_type=None, column_number=None, row_number=None):
    if(org_type=="city"):
        df = cities_2020_merged_sub
        subset = df[df['Question Number'] == question_number]
        answer = subset[subset["Account Number"] == account]
        if((column_number is not None) and (row_number is None)):
            answer = answer[answer["Column Number"] == column_number]
        elif((column_number is not None) and (row_number is not None)):
            answer = answer[(answer["Column Number"] == column_number) & (answer["Row Number"] == row_number)]
        elif((column_number is None) and (row_number is not None)):
            answer = answer[answer["Row Number"] == row_number]
        else:
            pass
    elif(org_type=="corp" and corp_res_type=="cc"):
        df = corporations_2020
        subset = df[df['question_number'] == question_number]
        answer = subset[subset["account_number"] == account]
        if((column_number is not None) and (row_number is None)):
            answer = answer[answer["column_number"] == column_number]
        elif((column_number is not None) and (row_number is not None)):
            answer = answer[(answer["column_number"] == column_number) & (answer["row_number"] == row_number)]
        elif((column_number is None) and (row_number is not None)):
            answer = answer[answer["row_number"] == row_number]
        else:
            pass
    elif(org_type=="corp" and corp_res_type=="ws"):
        df = all_corps_cc[year]
        subset = df[df['question_number'] == question_number]
        answer = subset[subset["account_number"] == account]
    else:
        print("Something went wrong. Try again.")
    
    return answer

In [ ]:

scores_lo_hi = {
    "Low" : 1, 
    "Medium Low": 2, 
    "Medium":3,
    "Medium High":4,
    "High":5,
    "Do not know":0.5,
    "Does not currently impact the city":0.5,
    "nan":0,
}

scores_increasing_decreasing = {
    "Increasing" : 2,
    "Decreasing" : 0.5,
    "None" : 1, 
    "Do not know" : 1, 
    "Not expected to happen in the future" : 0.1,
    "nan" : 0.0
}

def score_hazard_risk(cdf):
    prob = scores_lo_hi[str(cdf[cdf["Column Number"] == 3.0]["Response Answer"].iloc[0])]
    mag = scores_lo_hi[str(cdf[cdf["Column Number"] == 4.0]["Response Answer"].iloc[0])]
    
    future_freq = scores_increasing_decreasing[str(cdf[cdf["Column Number"] == 8.0]["Response Answer"].iloc[0])]
    future_intensity = scores_increasing_decreasing[str(cdf[cdf["Column Number"] == 9.0]["Response Answer"].iloc[0])]
    
    future_mag = scores_lo_hi[str(cdf[cdf["Column Number"] == 10.0]["Response Answer"].iloc[0])]
    return (prob*mag) + (prob*future_freq*future_intensity*future_mag)

def return_risk_and_score_per_county(hazard_type):
    risk = []
    self_reported_score = []

    for i in range(len(vulnerability)):
        df = fetch_answer_individual(vulnerability["Account Number"].iloc[i], '2.1')
        idx = np.where(np.array(df["Response Answer"]) == hazard_type)[0]
        if(len(idx) > 0):
            risk.append(1)
            cdf = df[df["Row Number"] == df["Row Number"].iloc[idx[0]]]
            self_reported_score.append(score_hazard_risk(cdf))
        else:
            risk.append(np.nan)
            self_reported_score.append(np.nan)
    return risk, np.nan_to_num(np.array(self_reported_score))

In [ ]:
risk_storm_surge, score_storm_surge = return_risk_and_score_per_county("Storm and wind > Storm surge")

risk_monsoon, score_monsoon = return_risk_and_score_per_county("Extreme Precipitation > Monsoon")

risk_rain_storm, score_rain_storm = return_risk_and_score_per_county("Extreme Precipitation > Rain storm")

risk_flash_flood, score_flash_flood = return_risk_and_score_per_county("Flood and sea level rise > Flash / surface flood")

risk_river_flood, score_river_flood = return_risk_and_score_per_county("Flood and sea level rise > River flood")

risk_coastal_flood, score_coastal_flood = return_risk_and_score_per_county("Flood and sea level rise > Coastal flood")

risk_groundwater_flood, score_groundwater_flood = return_risk_and_score_per_county("Flood and sea level rise > Groundwater flood")

risk_permanent_inundation, score_permanent_inundation = return_risk_and_score_per_county("Flood and sea level rise > Permanent inundation")


In [ ]:
flood_score = score_storm_surge + score_flash_flood + score_river_flood + score_coastal_flood + score_groundwater_flood + score_permanent_inundation
precip_score = score_rain_storm + score_monsoon

flood_score_ranked = standardize_rank(flood_score, direction=1)
precip_score_ranked = standardize_rank(precip_score, direction=1)

In [ ]:
exposure = pd.read_excel("../input/nfipexposure/02_NFIP_2018_By_Zip5_Exposure.xlsx", sheet_name="Exposure_ByZip", skiprows=5)


In [ ]:
insured_locations = []
for i in range(len(match_fips)):
    if(not flag_multi_counties[i]):
        fips = int(match_fips[i])
        cdf = exposure[exposure["StateCountyFIPS"] == fips]
        insured_locations.append(np.sum(cdf["Locations"])/population[i])
    else:
        fips = match_fips[i]
        temp = []
        for j in range(len(fips)):
            cdf = exposure[exposure["StateCountyFIPS"] == fips[j]]
            temp.append(np.sum(cdf["Locations"]))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        insured_locations.append(weighted_avg/population[i])


In [ ]:
insured_locations_ranked = standardize_rank(insured_locations, direction=1)

In [ ]:
fema = pd.read_csv("../input/femadisasters/DisasterDeclarationsSummaries.csv")
ct = np.char.zfill(np.array(fema['fipsCountyCode']).astype(str), 3)
st = np.char.zfill(np.array(fema['fipsStateCode']).astype(str), 2)
fema['fips'] = np.core.defchararray.add(st, ct).astype(int)

In [ ]:
fema_flood_events = []
for i in range(len(match_fips)):
    if(not flag_multi_counties[i]):
        fips = int(match_fips[i])
        cdf = fema[(fema["incidentType"] == "Flood") | ((fema["incidentType"] == "Severe Storm(s)") & (fema["declarationTitle"].str.lower().str.contains("flood")))]
        cdf = cdf[cdf["fips"] == fips]
        fema_flood_events.append(len(cdf))
    else:
        fips = match_fips[i]
        temp = []
        for j in range(len(fips)):
            cdf = fema[(fema["incidentType"] == "Flood") | ((fema["incidentType"] == "Severe Storm(s)") & (fema["declarationTitle"].str.lower().str.contains("flood")))]
            cdf = cdf[cdf["fips"] == fips[j]]
            temp.append(len(cdf))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        fema_flood_events.append(weighted_avg)

fema_coastal_storms = []
for i in range(len(match_fips)):
    if(not flag_multi_counties[i]):
        fips = int(match_fips[i])
        cdf = fema[(fema["incidentType"] == "Coastal Storm") | ((fema["incidentType"] == "Severe Storm(s)") & (fema["declarationTitle"].str.lower().str.contains("coastal")))]
        cdf = cdf[cdf["fips"] == fips]
        fema_coastal_storms.append(len(cdf))
    else:
        fips = match_fips[i]
        temp = []
        for j in range(len(fips)):
            cdf = fema[(fema["incidentType"] == "Coastal Storm") | ((fema["incidentType"] == "Severe Storm(s)") & (fema["declarationTitle"].str.lower().str.contains("coastal")))]
            cdf = cdf[cdf["fips"] == fips[j]]
            temp.append(len(cdf))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        fema_coastal_storms.append(weighted_avg)

fema_landslides = []
for i in range(len(match_fips)):
    if(not flag_multi_counties[i]):
        fips = int(match_fips[i])
        cdf = fema[(fema["incidentType"] == "Mud/Landslide") | ((fema["incidentType"] == "Severe Storm(s)") & (fema["declarationTitle"].str.lower().str.contains("landslide")))]
        cdf = cdf[cdf["fips"] == fips]
        fema_landslides.append(len(cdf))
    else:
        fips = match_fips[i]
        temp = []
        for j in range(len(fips)):
            cdf = fema[(fema["incidentType"] == "Mud/Landslide") | ((fema["incidentType"] == "Severe Storm(s)") & (fema["declarationTitle"].str.lower().str.contains("landslide")))]
            cdf = cdf[cdf["fips"] == fips[j]]
            temp.append(len(cdf))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        fema_landslides.append(weighted_avg)


In [ ]:
fema_flood_events_ranked = standardize_rank(fema_flood_events, direction=1)
fema_coastal_storms_ranked = standardize_rank(fema_coastal_storms, direction=1)
fema_landslides_ranked = standardize_rank(fema_landslides, direction=1)

In [ ]:
building = pd.read_csv("../input/housingyear/Year_Housing/Building_Year.csv", skiprows=1)
building.head()
building = building.loc[:, ~building.columns.str.startswith('Margin')]
building["id"] = building["id"].str[9:].astype(int)

In [ ]:
percent_built_before_1999 = []
for i in range(len(match_fips)):
    if(not flag_multi_counties[i]):
        fips = int(match_fips[i])
        cdf = building[building["id"] == fips]
        percent_built_before_1999.append(np.sum(np.array(cdf)[0][6:])/(np.array(cdf)[0][2]))
    else:
        fips = match_fips[i]
        temp = []
        for j in range(len(fips)):
            cdf = building[building["id"] == fips[j]]
            temp.append(np.sum(np.array(cdf)[0][6:])/(np.array(cdf)[0][2]))
        ar = np.array(temp)
        weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
        percent_built_before_1999.append(weighted_avg)


In [ ]:
percent_built_before_1999_ranked = standardize_rank(percent_built_before_1999, direction=1)

In [ ]:
nri = pd.read_csv(path + "NRI_Table_Counties.csv")

def nri_select(field):
    res = []
    for i in range(len(match_fips)):
        if(not flag_multi_counties[i]):
            fips = int(match_fips[i])
            cdf = nri[nri["STCOFIPS"] == fips]
            try:
                res.append(float(cdf[field]))
            except:
                res.append(np.nan)
        else:
            fips = match_fips[i]
            temp = []
            for j in range(len(fips)):
                cdf = nri[nri["STCOFIPS"] == fips[j]]
                try:
                    temp.append(float(cdf[field]))
                except:
                    temp.append(np.nan)
            ar = np.array(temp)
            weighted_avg = np.average(ar, weights=np.array(ast.literal_eval(counties_weights[i])))
            res.append(weighted_avg)
    return np.array(res)

coastal_flood_eals = nri_select("CFLD_EALS")
river_flood_eals = nri_select("RFLD_EALS")
landslide_eals = nri_select("LNDS_EALS")

In [ ]:
coastal_flood_eals_ranked = standardize_rank(coastal_flood_eals, direction=1)
river_flood_eals_ranked = standardize_rank(river_flood_eals, direction=1)
landslide_eals_ranked = standardize_rank(landslide_eals, direction=1)

In [ ]:
vulnerability_raw = vulnerability.copy()
vulnerability_ranked = vulnerability.copy()
vulnerability_aggr = vulnerability.copy()

In [ ]:
### EXPOSURE 


vulnerability_raw["flood_score"] = flood_score
vulnerability_raw["precip_score"] = precip_score
vulnerability_raw["insured_locations_normalized_by_pop"] = insured_locations

vulnerability_raw["fema_flood_events"] = fema_flood_events
vulnerability_raw["fema_coastal_storms"] = fema_coastal_storms
vulnerability_raw["fema_landslides"] = fema_landslides

### SENSITIVITY

vulnerability_raw["percent_built_before_1999"] = percent_built_before_1999
vulnerability_raw["coastal_flood_expected_annual_loss"] = coastal_flood_eals
vulnerability_raw["river_flood_expected_annual_loss"] = river_flood_eals
vulnerability_raw["landslide_expected_annual_loss"] = landslide_eals

vulnerability_raw.to_csv("vulnerability_cities_us_floods_kpis_raw.csv")

In [ ]:
### EXPOSURE 
vulnerability_ranked["flood_score_ranked"] = flood_score_ranked
vulnerability_ranked["precip_score_ranked"] = precip_score_ranked
vulnerability_ranked["insured_locations_ranked"] = insured_locations_ranked

vulnerability_ranked["fema_flood_events_ranked"] = fema_flood_events_ranked
vulnerability_ranked["fema_coastal_storms_ranked"] = fema_coastal_storms_ranked
vulnerability_ranked["fema_landslides_ranked"] = fema_landslides_ranked

### SENSITIVITY

vulnerability_ranked["percent_built_before_1999_ranked"] = percent_built_before_1999_ranked
vulnerability_ranked["coastal_flood_expected_annual_loss_ranked"] = coastal_flood_eals_ranked
vulnerability_ranked["river_flood_expected_annual_loss_ranked"] = river_flood_eals_ranked
vulnerability_ranked["landslide_expected_annual_loss_ranked"] = landslide_eals_ranked
vulnerability_ranked.to_csv("vulnerability_cities_us_floods_kpis_ranked.csv")


In [ ]:
def aggr_and_rescale(arr):
    avg_arr = np.nanmean(arr, axis=1)
    rescaled_avg_arr = standardize_rank(avg_arr)
    return rescaled_avg_arr

In [ ]:
all_expo_floods_kpis = np.c_[flood_score_ranked, precip_score_ranked, insured_locations_ranked, fema_flood_events_ranked,
                          fema_coastal_storms_ranked, fema_landslides_ranked]

aggr_expo_floods_kpis = aggr_and_rescale(all_expo_floods_kpis)

all_sens_floods_kpis = np.c_[percent_built_before_1999_ranked, coastal_flood_eals_ranked, river_flood_eals_ranked, landslide_eals_ranked]

aggr_sens_floods_kpis = aggr_and_rescale(all_sens_floods_kpis)

aggr_floods_kpi = aggr_and_rescale(np.c_[aggr_expo_floods_kpis, aggr_sens_floods_kpis])

In [ ]:
vulnerability_aggr = vulnerability.copy()

vulnerability_aggr["aggr_floods_kpis"] = aggr_floods_kpi
vulnerability_aggr["aggr_exposure_floods_kpis"] = aggr_expo_floods_kpis
vulnerability_aggr["aggr_sensitivity_floods_kpis"] = aggr_sens_floods_kpis

vulnerability_aggr.to_csv("vulnerability_cities_us_floods_aggr_kpis.csv")